# Module 7: Best Practices and Capstone Projects

**Duration:** ~35 minutes

**Learning Objectives:**
- Apply best practices for production-ready agents
- Understand common pitfalls and how to avoid them
- Review the complete agentic development workflow
- Choose and begin your capstone project

---

## 7.1 Best Practices Overview

Building production-ready agents requires attention to multiple areas:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    Agent Development Best Practices                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  ┌────────────┐  │
│  │   Prompt     │  │    Tool      │  │Architecture  │  │ Production │  │
│  │ Engineering  │  │   Design     │  │  Patterns    │  │   Ops      │  │
│  ├──────────────┤  ├──────────────┤  ├──────────────┤  ├────────────┤  │
│  │• Clear       │  │• Single      │  │• Separation  │  │• Monitoring│  │
│  │  instructions│  │  purpose     │  │  of concerns │  │• Logging   │  │
│  │• Examples    │  │• Type hints  │  │• Loose       │  │• Alerting  │  │
│  │• Constraints │  │• Docstrings  │  │  coupling    │  │• Scaling   │  │
│  │• Edge cases  │  │• Error       │  │• Idempotency │  │• Security  │  │
│  │              │  │  handling    │  │              │  │            │  │
│  └──────────────┘  └──────────────┘  └──────────────┘  └────────────┘  │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 7.2 Prompt Engineering for Agents

### Writing Effective Instructions

Agent instructions are different from regular prompts because:
1. They persist across multiple turns
2. They must work with tool calling
3. They guide autonomous decision-making

In [ ]:
# ❌ Bad: Vague instructions
bad_instruction = """
You are a helpful assistant. Help users with their questions.
"""

# ✅ Good: Clear, structured instructions
good_instruction = """
You are a customer support specialist for TechCorp's software products.

## Your Role
Help customers troubleshoot issues, answer product questions, and escalate
complex problems to human agents when necessary.

## Available Tools
- search_knowledge_base: Search documentation for answers
- get_customer_info: Retrieve customer account details
- create_ticket: Escalate issues to human support
- check_system_status: Check for known outages

## Guidelines
1. Always verify customer identity before accessing account info
2. Search the knowledge base before saying "I don't know"
3. For billing issues, create a ticket (you cannot modify billing)
4. Use simple, non-technical language
5. If the customer seems frustrated, acknowledge their feelings

## Escalation Criteria
Create a support ticket when:
- Issue requires accessing internal systems you can't reach
- Customer explicitly requests human support
- Problem persists after 3 troubleshooting attempts
- Security-related concerns

## Response Format
- Be concise but thorough
- Use numbered steps for procedures
- End with a clear next action or question
"""

print("❌ Bad Instruction:")
print(bad_instruction)
print("\n" + "="*60 + "\n")
print("✅ Good Instruction:")
print(good_instruction)

In [ ]:
# Instruction template pattern

instruction_template = """
You are a {role} for {company/domain}.

## Core Responsibilities
{list of main tasks}

## Available Tools
{tool name}: {when and how to use it}
...

## Decision Framework
When {situation}, do {action}
When {other situation}, do {other action}
...

## Constraints
- Never {prohibited action}
- Always {required behavior}
- Limit {bounded action} to {limit}

## Communication Style
{tone, format, language preferences}

## Escalation/Handoff
{when and how to escalate}
"""

print("📝 Instruction Template:")
print(instruction_template)

## 7.3 Tool Design Best Practices

### The SOLID Principles for Tools

| Principle | Application to Tools |
|-----------|--------------------|
| **S**ingle Responsibility | One tool = one action |
| **O**pen/Closed | Tools should be extensible |
| **L**iskov Substitution | Consistent return types |
| **I**nterface Segregation | Small, focused tool sets |
| **D**ependency Inversion | Tools depend on abstractions |

In [ ]:
from typing import Dict, Any, List, Optional

# ❌ Bad: Tool does too much, unclear naming, poor documentation
def do_stuff(data: dict) -> dict:
    """Does stuff with data."""
    # What kind of stuff? Nobody knows!
    pass

# ✅ Good: Single purpose, clear naming, comprehensive documentation
def search_products(
    query: str,
    category: Optional[str] = None,
    max_results: int = 10,
    min_price: Optional[float] = None,
    max_price: Optional[float] = None
) -> Dict[str, Any]:
    """
    Search the product catalog for items matching the query.
    
    Use this tool when the user wants to find products. The search
    matches against product names, descriptions, and tags.
    
    Args:
        query: Search terms (e.g., "wireless headphones", "laptop stand")
        category: Filter by category (e.g., "Electronics", "Furniture")
        max_results: Maximum number of results (1-50, default 10)
        min_price: Minimum price filter in USD
        max_price: Maximum price filter in USD
    
    Returns:
        Dictionary containing:
        - status: "success" or "error"
        - count: Number of results found
        - products: List of matching products with id, name, price, category
        - message: Error message if status is "error"
    
    Example:
        >>> search_products("headphones", category="Electronics", max_price=100)
        {"status": "success", "count": 3, "products": [...]}
    """
    # Implementation...
    return {"status": "success", "count": 0, "products": []}

print("✅ Well-designed tool with proper documentation")

In [ ]:
# Error handling pattern for tools

def robust_tool_template(
    required_param: str,
    optional_param: Optional[str] = None
) -> Dict[str, Any]:
    """
    Template for a robust tool with proper error handling.
    
    Args:
        required_param: A required parameter
        optional_param: An optional parameter
    
    Returns:
        Standardized response dictionary
    """
    # Input validation
    if not required_param or not required_param.strip():
        return {
            "status": "error",
            "error_code": "INVALID_INPUT",
            "message": "required_param cannot be empty"
        }
    
    try:
        # Main logic
        result = do_something(required_param)
        
        # Success response
        return {
            "status": "success",
            "data": result,
            "message": "Operation completed successfully"
        }
        
    except ConnectionError as e:
        # Specific error handling
        return {
            "status": "error",
            "error_code": "CONNECTION_FAILED",
            "message": f"Could not connect to service: {str(e)}",
            "retry": True  # Hint to agent that retry might help
        }
        
    except Exception as e:
        # Generic error handling
        return {
            "status": "error",
            "error_code": "UNKNOWN_ERROR",
            "message": f"An unexpected error occurred: {str(e)}",
            "retry": False
        }

def do_something(param):
    return {"processed": param}

print("📝 Robust tool template with error handling")

## 7.4 Architecture Patterns

### Pattern 1: Router-Worker

A router agent classifies requests and delegates to specialized workers.

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import FunctionTool

# Router-Worker Pattern

# Worker agents with narrow focus
billing_worker = Agent(
    name="billing_specialist",
    model="gemini-2.0-flash",
    description="Handles billing inquiries, payment issues, and invoices",
    instruction="You specialize in billing. Only handle billing-related questions."
)

technical_worker = Agent(
    name="technical_specialist", 
    model="gemini-2.0-flash",
    description="Handles technical issues, troubleshooting, and how-to questions",
    instruction="You specialize in technical support. Only handle technical questions."
)

# Router agent
router_agent = Agent(
    name="support_router",
    model="gemini-2.0-flash",
    description="Routes customer requests to appropriate specialists",
    instruction="""
    Classify incoming requests and route to the appropriate specialist:
    - billing_specialist: Payment, invoices, subscriptions, refunds
    - technical_specialist: Bugs, errors, how-to, configuration
    
    If unclear, ask a clarifying question.
    """,
    sub_agents=[billing_worker, technical_worker]
)

print("✅ Router-Worker pattern implemented")

In [ ]:
# Pattern 2: Pipeline with Checkpoints

from google.adk.agents import SequentialAgent

def validate_data(data: Dict[str, Any]) -> Dict[str, Any]:
    """Validate input data before processing."""
    errors = []
    if not data.get("email"):
        errors.append("Email is required")
    if not data.get("amount"):
        errors.append("Amount is required")
    
    return {
        "status": "valid" if not errors else "invalid",
        "errors": errors,
        "data": data
    }

validator_agent = Agent(
    name="validator",
    model="gemini-2.0-flash",
    description="Validates input data",
    instruction="Validate the data and report any issues.",
    tools=[FunctionTool(validate_data)]
)

processor_agent = Agent(
    name="processor",
    model="gemini-2.0-flash",
    description="Processes validated data",
    instruction="Process the validated data. Skip if validation failed."
)

confirmer_agent = Agent(
    name="confirmer",
    model="gemini-2.0-flash",
    description="Sends confirmation",
    instruction="Generate a confirmation message for the user."
)

pipeline = SequentialAgent(
    name="order_pipeline",
    description="Order processing pipeline",
    sub_agents=[validator_agent, processor_agent, confirmer_agent]
)

print("✅ Pipeline with checkpoints pattern implemented")

## 7.5 Common Pitfalls and Solutions

### Pitfall 1: Tool Overload

In [ ]:
# ❌ Bad: Too many tools confuse the agent
overloaded_agent_tools = [
    "search_products", "get_product", "list_products", "filter_products",
    "sort_products", "count_products", "get_product_reviews", "search_reviews",
    "create_order", "update_order", "cancel_order", "get_order", "list_orders",
    "add_to_cart", "remove_from_cart", "get_cart", "clear_cart", "checkout",
    "get_user", "update_user", "get_preferences", "set_preferences"
]  # 22 tools - too many!

print(f"❌ Bad: {len(overloaded_agent_tools)} tools in one agent")

# ✅ Good: Split into focused agents
product_agent_tools = ["search_products", "get_product_details"]  # 2 tools
order_agent_tools = ["create_order", "get_order_status", "cancel_order"]  # 3 tools
cart_agent_tools = ["add_to_cart", "view_cart", "checkout"]  # 3 tools

print(f"✅ Good: Split into agents with {len(product_agent_tools)}, {len(order_agent_tools)}, {len(cart_agent_tools)} tools each")

In [ ]:
# Pitfall 2: Unbounded Tool Calls

# ❌ Bad: No limits on results
def bad_search(query: str) -> Dict[str, Any]:
    """Search everything."""
    # Could return millions of results!
    results = database.search(query)  # No limit
    return {"results": results}

# ✅ Good: Bounded results with pagination
def good_search(
    query: str,
    limit: int = 10,
    offset: int = 0
) -> Dict[str, Any]:
    """
    Search with pagination.
    
    Args:
        query: Search terms
        limit: Max results (1-100, default 10)
        offset: Starting position for pagination
    """
    # Enforce limits
    limit = min(max(1, limit), 100)
    
    results = database.search(query, limit=limit, offset=offset)
    total = database.count(query)
    
    return {
        "results": results,
        "limit": limit,
        "offset": offset,
        "total": total,
        "has_more": offset + limit < total
    }

print("✅ Always bound your queries!")

In [ ]:
# Pitfall 3: Ignoring Context Window Limits

# ❌ Bad: Returns entire document
def bad_get_document(doc_id: str) -> Dict[str, Any]:
    """Get full document content."""
    doc = database.get_document(doc_id)
    return {"content": doc.full_text}  # Could be 100k+ tokens!

# ✅ Good: Returns summary with option for sections
def good_get_document(
    doc_id: str,
    section: Optional[str] = None,
    max_chars: int = 5000
) -> Dict[str, Any]:
    """
    Get document content with context-aware truncation.
    
    Args:
        doc_id: Document ID
        section: Specific section to retrieve (None for summary)
        max_chars: Maximum characters to return
    
    Returns:
        Document metadata and content (truncated if necessary)
    """
    doc = database.get_document(doc_id)
    
    if section:
        content = doc.get_section(section)[:max_chars]
    else:
        content = doc.summary[:max_chars]
    
    return {
        "id": doc_id,
        "title": doc.title,
        "sections": doc.section_names,  # So agent can request specific sections
        "content": content,
        "truncated": len(doc.full_text) > max_chars
    }

print("✅ Be mindful of context window limits!")

## 7.6 Testing Strategies

In [ ]:
# Testing template for agents

testing_template = '''
# tests/test_agent.py
import pytest
from unittest.mock import Mock, patch
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner

from my_agent.agent import root_agent
from my_agent.tools import search_products


# Unit test for tools
class TestTools:
    def test_search_products_with_results(self):
        """Test search returns expected results."""
        result = search_products("laptop")
        
        assert result["status"] == "success"
        assert "products" in result
        assert isinstance(result["products"], list)
    
    def test_search_products_empty_query(self):
        """Test search handles empty query gracefully."""
        result = search_products("")
        
        assert result["status"] == "error"
        assert "message" in result
    
    def test_search_products_no_results(self):
        """Test search with no matching results."""
        result = search_products("xyznonexistent123")
        
        assert result["status"] == "success"
        assert result["count"] == 0


# Integration test for agent
class TestAgentIntegration:
    @pytest.fixture
    async def runner(self):
        """Create test runner."""
        return InMemoryRunner(agent=root_agent)
    
    @pytest.mark.asyncio
    async def test_agent_uses_search_tool(self, runner):
        """Test agent correctly uses search tool for product queries."""
        session = await runner.session_service.create_session(
            app_name="test", user_id="test_user"
        )
        
        response = await runner.run(
            user_id="test_user",
            session_id=session.id,
            new_message="Find me laptops under $1000"
        )
        
        # Collect response
        text = ""
        async for event in response:
            if hasattr(event, "content") and event.content:
                for part in event.content.parts:
                    if hasattr(part, "text"):
                        text += part.text
        
        # Verify response mentions products
        assert "laptop" in text.lower() or "product" in text.lower()


# Behavioral test scenarios
class TestAgentBehavior:
    test_scenarios = [
        {
            "input": "What products do you have?",
            "expected_tool": "search_products",
            "expected_in_response": ["product"]
        },
        {
            "input": "Help me with my order",
            "expected_tool": "get_order_status",
            "expected_in_response": ["order"]
        }
    ]
    
    @pytest.mark.parametrize("scenario", test_scenarios)
    @pytest.mark.asyncio
    async def test_scenario(self, scenario):
        """Test agent handles various scenarios correctly."""
        # Implementation...
        pass
'''

print("📝 Testing Template:")
print(testing_template)

## 7.7 Complete Workflow Summary

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    Agentic Development Workflow                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  1. DESIGN                                                               │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ • Define agent responsibilities                                  │    │
│  │ • Identify required tools (MCP vs FunctionTool)                 │    │
│  │ • Plan multi-agent architecture (A2A connections)               │    │
│  │ • Write Agent Cards for A2A exposure                            │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                              │                                           │
│                              ▼                                           │
│  2. IMPLEMENT                                                            │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ • Create tools with proper types and docs                        │    │
│  │ • Write agent instructions                                       │    │
│  │ • Configure MCP connections                                      │    │
│  │ • Set up RemoteA2aAgent connections                             │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                              │                                           │
│                              ▼                                           │
│  3. TEST                                                                 │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ • Unit test tools                                                │    │
│  │ • Integration test agent                                         │    │
│  │ • Test with ADK web UI (adk web)                                │    │
│  │ • Run evaluation suite                                           │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                              │                                           │
│                              ▼                                           │
│  4. DEPLOY                                                               │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ • Deploy to Agent Engine or Cloud Run                            │    │
│  │ • Configure sessions and memory                                  │    │
│  │ • Set up CI/CD pipeline                                          │    │
│  │ • Enable monitoring and alerting                                 │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                              │                                           │
│                              ▼                                           │
│  5. OPERATE                                                              │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ • Monitor metrics and logs                                       │    │
│  │ • Review conversation traces                                     │    │
│  │ • Collect user feedback                                          │    │
│  │ • Iterate on instructions and tools                              │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## 7.8 Capstone Projects

Choose one of the following capstone projects to complete over the next week. Each project incorporates all the concepts from this workshop.

### Option A: Currency Exchange Multi-Agent System

Build a complete currency exchange system following the Google Codelab pattern.

**Requirements:**
- Exchange Rate Agent: Fetches real-time rates via MCP
- Conversion Agent: Handles currency conversions
- History Agent: Tracks conversion history
- Orchestrator: Routes requests and provides recommendations

**Technical Components:**
- A2A communication between agents
- MCP connection to exchange rate API
- Session management for user preferences
- Deployment to Cloud Run

---

### Option B: Customer Service Platform

Build an enterprise customer service system.

**Requirements:**
- Triage Agent: Classifies and routes requests
- Product Agent: Answers product questions
- Order Agent: Handles order inquiries
- Escalation Agent: Creates tickets for human review

**Technical Components:**
- MCP connection to knowledge base
- A2A for agent coordination
- Memory Bank for customer context
- Evaluation suite with test cases

---

### Option C: Research Assistant

Build an intelligent research assistant.

**Requirements:**
- Search Agent: Web and document search
- Summarizer Agent: Condenses findings
- Citation Agent: Tracks and formats sources
- Report Agent: Generates research reports

**Technical Components:**
- MCP for search and document access
- A2A pipeline for research workflow
- Artifact generation (reports)
- Quality evaluation metrics

## 7.9 Capstone Evaluation Criteria

Your capstone will be evaluated on:

| Criteria | Points | Description |
|----------|--------|-------------|
| **Architecture** | 25 | Clean separation, appropriate use of A2A/MCP |
| **Code Quality** | 20 | Type hints, documentation, error handling |
| **Functionality** | 25 | Agents work correctly, tools are effective |
| **Testing** | 15 | Unit tests, integration tests, evaluation |
| **Documentation** | 15 | README, Agent Cards, inline comments |

### Submission Requirements

1. GitHub repository with complete code
2. README with setup instructions
3. Agent Cards for all A2A-exposed agents
4. Test suite with >80% coverage
5. Demo video (2-3 minutes) showing the system in action

## 7.10 Getting Started with Your Capstone

Navigate to the `capstone-projects/` folder in this repository and choose your project:

```
capstone-projects/
├── option_a_currency_exchange/
│   ├── README.md              # Detailed requirements
│   ├── starter_code/          # Boilerplate to get started
│   └── solution_hints/        # Hints if you get stuck
│
├── option_b_customer_service/
│   ├── README.md
│   ├── starter_code/
│   └── solution_hints/
│
└── option_c_research_assistant/
    ├── README.md
    ├── starter_code/
    └── solution_hints/
```

---

## Workshop Summary

Congratulations on completing the **Agentic Development with Google Ecosystem** workshop!

### What You Learned

1. **Module 1**: Environment setup and Google's agentic stack
2. **Module 2**: ADK fundamentals, tools, and multi-agent systems
3. **Module 3**: MCP integration for external tools
4. **Module 4**: A2A protocol for agent collaboration
5. **Module 5**: Combining A2A and MCP in real architectures
6. **Module 6**: Production deployment with Vertex AI
7. **Module 7**: Best practices and capstone project

### Key Takeaways

- **ADK** is your framework for building agents
- **MCP** connects agents to external tools and data
- **A2A** enables agents to collaborate
- **Vertex AI** provides production infrastructure
- Good **prompt engineering** and **tool design** are critical

### Continue Learning

- [ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol](https://a2a-protocol.org/)
- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/docs/agent-builder/agent-engine)
- [Google Cloud Codelabs](https://codelabs.developers.google.com/?cat=ai)

---

**Good luck with your capstone project!** 🚀